In [1]:
from curl_cffi import requests as cureq
import pandas as pd
import time
import json
from rich import print
import os

In [6]:
response = cureq.get("https://www.sofascore.com/api/v1/unique-tournament/1644/season/7752/events/round/1", impersonate="chrome")

In [5]:
alldata =[]
games = json.loads(response.text)
for game in games["events"]:
    matchdata = {
                "tournament" : game["tournament"]["uniqueTournament"]["name"],
                "season" : game["season"]["year"],
                "round" : game["roundInfo"]["round"],
                "matchID" : game["id"],
                "matchCustomID" : game["customId"],
                "matchStatus" : game["status"]["description"], # Ended, Postponed, Not started
                "homeTeamnName" :game["homeTeam"]["name"],
                "homeTeamnNameCode":game["homeTeam"]["nameCode"],
                'homeTeamID' : game["homeTeam"]["id"],
                "awayTeamName": game["awayTeam"]["name"],
                'awayTeamNameCode': game["awayTeam"]["nameCode"],
                'awayTeamID' : game["awayTeam"]["id"],
    }
    
    if matchdata["matchStatus"] == "Ended":
        matchScores = {
            #'homeScoreHT' : game["homeScore"]["period1"],
            #'awayScoreHT' : game["awayScore"]["period1"],
            'homeScoreFT' : game["homeScore"]["normaltime"],
            'awayScoreFT' : game["awayScore"]["normaltime"],
        }
    else:
        matchScores = {
            #'homeScoreHT' : "-",
            #'awayScoreHT' : "-",
            'homeScoreFT' : "-",
            'awayScoreFT' : "-",
        }
    #data = matchdata | matchScores
    alldata.append(matchdata | matchScores)


In [56]:
os.makedirs('Season-2014', exist_ok=True)

In [4]:
df = pd.DataFrame(alldata)
df#.to_csv('Season-2014/round1.csv')

,tournament,season,round,matchID,matchCustomID,matchStatus,homeTeamnName,homeTeamnNameCode,homeTeamID,awayTeamName,awayTeamNameCode,awayTeamID,homeScoreFT,awayScoreFT
0,Kenyan Premier League,2014,1,5022474,JufbsNufb,Ended,Muhoroni Youth FC,MUY,138484,Sofapaka,SOF,138488,1,5
1,Kenyan Premier League,2014,1,5022472,PufbsUufb,Ended,Sony Sugar,SSU,138490,Top Fry Nakuru,TFN,138494,3,0
2,Kenyan Premier League,2014,1,5022476,BufbsYufb,Ended,Ulinzi Stars,ULS,138498,Chemelil Sugar,CSU,138476,1,1
3,Kenyan Premier League,2014,1,5022470,Hufbsavfb,Ended,Western Stima FC,WST,138500,Mathare United,MAU,138482,1,0
4,Kenyan Premier League,2014,1,5022478,zufbsFufb,Ended,Bandari FC,BAN,138474,Kenya Rev. Authority,KAU,138480,2,0
5,Kenyan Premier League,2014,1,5022480,RufbsWufb,Ended,Thika United,TUN,138492,Tusker,TUS,138496,1,1
6,Kenyan Premier League,2014,1,5022484,IDFsDufb,Ended,Kenya Commercial Bank,KCB,138478,AFC Leopards SC,LDS,76433,1,1
7,Kenyan Premier League,2014,1,5022482,gLdbsLufb,Ended,Gor Mahia FC,GOR,134306,Nairobi City Stars FC,NCS,138486,2,0


In [69]:
seasonIDs = {7752:"2014", 9841:"2015", 11265:"2016", 12921:"2017", 15858:"2018", 19876:"2018/2019", 24023:"2019/2020",
             34876:"2020/2021", 38844:"2021/2022", 45686:"2022/2023", 53922:"2023/2024", 65071:"2024/2025"}
class matches:
    def __init__(self):
        self.IDs={7752:"2014", 9841:"2015", 11265:"2016", 12921:"2017", 15858:"2018", 19876:"2018/2019", 24023:"2019/2020",
             34876:"2020/2021", 38844:"2021/2022", 45686:"2022/2023", 53922:"2023/2024", 65071:"2024/2025"}
        self.seasonIDs= [seasonID for seasonID in self.IDs]
        self.rounds = [round for round in range(1,33)]
        self.by_rounds(seasonIDs, round)
        self.by_date()

    def by_rounds(self, seasonID, seasonIDs):
        os.makedirs(f'Season-{self.IDs[{self.seasonID}]}', exist_ok=True)
        def extract():
            response = cureq.get("f'https://www.sofascore.com/api/v1/unique-tournament/1644/season/{seasonID}/events/round/{round}'",
                                  impersonate="chrome")
            return response
        def transform(response):
            round_data = []
            games = json.loads(response.text)
            for game in games["events"]:
                matchdata = {
                "tournament" : game["tournament"]["uniqueTournament"]["name"],
                "season" : game["season"]["year"],
                "round" : game["roundInfo"]["round"],
                "matchID" : game["id"],
                "matchCustomID" : game["customId"],
                "matchStatus" : game["status"]["description"], # Ended, Postponed, Not started
                "homeTeamnName" :game["homeTeam"]["name"],
                "homeTeamnNameCode":game["homeTeam"]["nameCode"],
                'homeTeamID' : game["homeTeam"]["id"],
                "awayTeamName": game["awayTeam"]["name"],
                'awayTeamNameCode': game["awayTeam"]["nameCode"],
                'awayTeamID' : game["awayTeam"]["id"],
                }
    
            if matchdata["matchStatus"]== "Ended":
                matchScores = {
                    'homeScoreHT' : game["homeScore"]["period1"],
                    'awayScoreHT' : game["awayScore"]["period1"],
                    'homeScoreFT' : game["homeScore"]["normaltime"],
                    'awayScoreFT' : game["awayScore"]["normaltime"]
                },
            else:
                matchScores = {
                    'homeScoreHT' : "-",
                    'awayScoreHT' : "-",
                    'homeScoreFT' : "-",
                    'awayScoreFT' : "-"
                }
            round_data.append(matchdata)
            return round_data       

        def load(round_data, seasonID, seasonIDs):
            df = pd.DataFrame(round_data)
            path=f"Season-2014/round-1"
            df.to_csv(f'{path}.csv')
            
        getdata=extract()
        process = transform(getdata)
        for seasonID in seasonIDs:
            load(process, seasonID, seasonIDs)
            
data = matches()
    

AttributeError: 'matches' object has no attribute 'seasonID'